Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [16]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# 1. Create Label Map

In [17]:
labels = [
    {'name':'a', 'id':1}, 
    {'name':'b', 'id':2},
    {'name':'c', 'id':3},
    {'name':'d', 'id':4},
    {'name':'e', 'id':5},
    {'name':'f', 'id':6},
    {'name':'g', 'id':7},
    {'name':'h', 'id':8},
    {'name':'i', 'id':9},
    {'name':'j', 'id':10},
    {'name':'k', 'id':11},
    {'name':'l', 'id':12},
    {'name':'m', 'id':13},
    {'name':'n', 'id':14},
    {'name':'o', 'id':15},
    {'name':'p', 'id':16},
    {'name':'q', 'id':17},
    {'name':'r', 'id':18},
    {'name':'s', 'id':19},
    {'name':'t', 'id':20},
    {'name':'u', 'id':21},
    {'name':'v', 'id':22},
    {'name':'w', 'id':23},
    {'name':'x', 'id':24},
    {'name':'y', 'id':25},
    {'name':'z', 'id':26},
] #verrà creata una "mappa" con le nostre classi

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n') #qui scrive il tutto sul file che verrà creato

# 2. Create TF records

In [18]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [6]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
Updating files:  33% (746/2250)
Updating files:  34% (765/2250)
Updating files:  35% (788/2250)
Updating files:  36% (810/2250)
Updating files:  37% (833/2250)
Updating files:  38% (855/2250)
Updating files:  39% (878/2250)
Updating files:  40% (900/2250)
Updating files:  41% (923/2250)
Updating files:  42% (945/2250)
Updating files:  43% (968/2250)
Updating files:  44% (990/2250)
Updating files:  45% (1013/2250)
Updating files:  46% (1035/2250)
Updating files:  47% (1058/2250)
Updating files:  48% (1080/2250)
Updating files:  49% (1103/2250)
Updating files:  50% (1125/2250)
Updating files:  51% (1148/2250)
Updating files:  51% (1163/2250)
Updating files:  52% (1170/2250)
Updating files:  53% (1193/2250)
Updating files:  54% (1215/2250)
Updating files:  55% (1238/2250)
Updating files:  56% (1260/2250)
Updating files:  57% (1283/2250)
Updating files:  58% (1305/2250)
Updating files:  59% (1328/2250)
Updating files:  60% (1350/2250)
Updating files:  61% (1373/225

In [6]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [19]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' #associerà al path questa cartella

In [5]:
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME} #creerà quella cartella
!copy PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config' MODEL_PATH+'/'+CUSTOM_MODEL_NAME #verrà copiato il file presente il pre-trained model (pipeline.config in quella cartella in model)

Sottodirectory o file Tensorflow\workspace\models\my_ssd_mobnet gi… esistente.
Errore occorso durante l'elaborazione: Tensorflow\workspace\models\my_ssd_mobnet.


Impossibile trovare il file specificato.


# 5. Update Config For Transfer Learning

In [20]:
import tensorflow as tf #importiamo tensorflow, per la nostra rete
#dalla libreria object_detection importiamo altri file per la riuscita del programma
from object_detection.utils import config_util 
from object_detection.protos import pipeline_pb2
#stessa cosa di prima, ma stavolta da un'altra libreria
from google.protobuf import text_format

In [21]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config' #setta il path del file pipeline.config

In [22]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH) #inizializziamo config

In [23]:
config #questa è la funzione base che verrà modificata

{'model': ssd {
   num_classes: 26
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 4e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.01
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.997
         scale: true
         epsilon: 0.001
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
   

In [24]:
#viene creata una sorta di shell config dove andare a modificare
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig() #viene creato una sorta di template della pipeline
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [25]:
#aggiorna il file pipeline.config
pipeline_config.model.ssd.num_classes = 26 #da modificare
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [26]:
#qui effettivamente viene aggiornato il nostro file pipeline.config con ciò che è stato scritto nel box precedente
config_text = text_format.MessageToString(pipeline_config) #il messaggio viene convertito in stringa da scrivere                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [15]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=1000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME)) #qui si può modificare il numero di steps

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=1000


# 7. Load Train Model From Checkpoint

In [29]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [30]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-4')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [31]:
import cv2 
import numpy as np
import tensorflow as tf
from time
import os

In [10]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [ ]:
cap.release()

In [ ]:
cap = cv2.VideoCapture(0) #utilizzo di opencv per inizializzare la webcam predefinita del dispositivo
history = [] #creazione di un array che servirà per far uscire in output ciò che identifica
word_old="" #creazione di una variabile d'appoggio per fare i confronti
#i=1 #creazione di un contatore
'''
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
'''

In [13]:
while True: #per sempre
    ret, frame = cap.read() #cattura il frame della webcam
    image_np = np.array(frame) #convertiamo quel frame in un vettore numpy
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32) #convertiamo il tutto in tensorflow tensor
    detections = detect_fn(input_tensor) #avviene l'identificazione utilizzando la funzione
    
    num_detections = int(detections.pop('num_detections')) #abbiamo un numero di indentificazioni per darlo in pasto alla prossima istruzione
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections #viene identificato il num_detections

    # detection_classes deve essere intero
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64) #viene identificato il num_classes

    label_id_offset = 1 #specifichiamo il primo indice della label
    image_np_with_detections = image_np.copy()

    #visualizza a schermo il box, la classe identificata e la percentuale di riconoscimento, una sorta di funzione a cui passiamo vari valori
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True, #usiamo le coordinate normali, per avere una corretta identificazione
                max_boxes_to_draw=1, #numero di box disegnati
                min_score_thresh=0.01, #soglia minima di riconoscimento, si può cambiare
                agnostic_mode=False)

    cv2.imshow('Silent',  cv2.resize(image_np_with_detections, (800, 600))) #mostro a video la finestra che mi mostra ciò che la mia webcam sta vedendo in quel momento e ridimensiono il tutto
    #capturing detection history
    word = category_index[detections['detection_classes'][np.argmax(detections['detection_scores'])]+1]['name'] #trasforma il numpy.array in una stringa
    if cv2.waitKey(1) & 0xFF == ord('w') : #se premiamo la lettere w(write) metterà in append e scriverà la parola trovata
        history.append(word) #mette in append quella stringa con il vettore
        print(history) #stampo il risultato
    #if word_old != word and i != 1: history.append(word) #mette in append quella stringa con il vettore, se la parola nuova è diversa da quella vecchia e se l'indice è maggiore di 1, in modo da evitare di prendere la prima foto
    #if i != 1: word_old = word #se la i è diversa da 1, usa la variabile d'appoggio
    #time.sleep(2.5) #tempo di pausa tra un segno e l'altro, si può cambiare
    #i=i+1 #incremento del contatore

    if cv2.waitKey(1) & 0xFF == ord('q') : #se qualcosa va male possiamo premere il tasto di interruzione, in questo caso q (quit)
        cap.release() #rilasciamo la webcam alla fine dell'esecuzone del programma
        break #interruzione del programma
parola = open("parola_trovata.txt", "w")
history = "".join(history)
parola.write(history)
parola.close()
cap.release()#rilasciamo la webcam alla fine dell'esecuzone del programma
'''
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=3,
                min_score_thresh=.90,
                agnostic_mode=False
    )

    cv2.imshow('Silent',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break
'''

KeyboardInterrupt: 